In [1]:
import pandahouse as ph
import pandas as pd

## Задание 2 / Task 2.
### Задание 2.1 / Task 2.1:
### Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.
### Write an optimal query that would return information about the number of very hardworking students. NB! Hardworking students are students that have successfuly solved 20 tasks (peas) within current month.


### Задание 2.2:
### Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.
### Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
- ARPU считается относительно всех пользователей, попавших в группы.

### An educational platform offers students courses with a "trial" model: a student can only solve 30 peas per day for free. In order to get an unlimited numer of tasks within a surtain subject the student must purchase the full course. A team has conducted an experimrnt, where they tested a new payment screen.
### Write one query that would get the following information about groups of users:
- ARPU
- ARPAU
- CR into purchase
- CR of an active user into purchase
- CR of an active math course user (subject = ’math’) into purchase of the math course
- ARPU must be calculated according to all users within their group

### Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах / An active user is a user that has solved successfuly more than 10 tasks within all subjects.

### Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике / An active math course user is a user that has solved successfuly more than 2 tasks in subject 'math'.

## Решение / Solution

### Задание 2.1 / Task 2.1
- Так как все данные источника данных находятся в пределах одного месяца, просто создадим подзапрос, который будет считать количество правильно решенных заданий, сгруппированных по ученикам. / Because all data from the source is given within the same month, one can just create a sub-query that would count the number of correctly solved tasks, grouped by student ids.
- Далее в основном запросе зададим дополнительное условие, где количество решенных заданий будет больше 20 и посчитаем количество студентов, которые удовлетворяют этому условию. / Then in the main query one can pose an additional condition, that the nuber of correctly solved tasks must be greate than 20 and count the nuber of students that fit into that condition

In [2]:
#объявляем параметры подключения
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
query_2_1 = '''
Select uniqExact(st_id) AS num_of_hardworking_students
FROM
    (Select 
        st_id as st_id,
        count(correct) as num_of_peas
    
    From default.peas
    WHERE correct = 1
    GROUP BY st_id
    )
Where num_of_peas > 20
'''
task_2_1 = ph.read_clickhouse(query_2_1, connection=connection_default)
task_2_1

,num_of_hardworking_students
0,130


### Задание 2.2 / Task 2.2
- ARPU = Income / Users
- ARPAU = Income / active Users
- CR = converted users / all users

### Так как значения, с которыми необходимо оперировать для вычисления метрики в ClickHouse являются скалярными (в одну строку), имеет смысл использовать WITH при созаднии запроса, так как таким образом, можно обращаться к разным источникам и получать оттуда нужные нам значения для расчета метрик.
### Поскольку в задании необходимо определить метрики для каждой из групп (pilot, control), необходимо к каждому из использованных источников данных добавлять информацию из default.studs.
### Используем ранее созданный запрос (задание 2.1), предварительно модифицировав его согласно новому заданию, для определения количеств активных студентов.
--------------------------------------------------------------------------------------------------------------------------------
### Because of the fact that values, used to calculate the required metrics are considered scalar in Clickhouse (in one row), it seems sensible to use WITH, while writing the query. That way, one can use different storages within one query.
### Because the task requires to calculate metrics according to different groups (pilot, control), one must add information from default.studs to each of the used storages
### Let us use a modified version of the earlier written query (task 2.1) in order to count active users

### Описание объявленных через WITH параметров / Description of parameters, created using WITH:

- pilot_studs - все студенты pilot группы (для pilot_ARPU и pilot_CR) / all students from the pilot group (for pilot_ARPU and pilot_CR)
- control_studs - все студенты control группы (для control_ARPU и control_CR) / all students from the control group (for control_ARPU and control_CR)
- pilot_active_studs - активные студенты pilot группы (для pilot_ARPAU и pilot_active_CR) / active students from the pilot group (for pilot_ARPAU and pilot_active_CR)
- control_active_studs - активные студенты control группы (для control_ARPAU и control_active_CR) / active students from the control group (for control_ARPAU and control_active_CR)
- pilot_active_math_studs - активные студенты по курсу математики pilot группы (для math_pilot_CR) / active math students from the pilot group (for math_pilot_CR)
- control_active_math_studs - активные студенты по курсу математики control группы (для math_control_CR) / active math students from the control group (for math_control_CR)
- paying_math_studs - количество студентов, конвертировавшихся в покупку курса по математике (для math_pilot_CR и math_control_CR) / number of students converted into buying the math course (for math_pilot_CR and math_control_CR)

In [21]:
query_2_2 = '''
WITH
    (Select uniqExact(st_id) From default.studs Where test_grp = 'pilot') AS all_pilot_users,
    (Select uniqExact(st_id) From default.studs Where test_grp = 'control') AS all_control_users,
    (Select sum(money) From default.final_project_check) AS full_rev,
    (Select uniqExact(st_id) From (Select st_id, sum(correct) AS num_of_peas From default.peas Group by st_id) Where num_of_peas > 10) AS all_active_studs,
    (Select uniqExact(st_id) From default.final_project_check AS l Join default.studs AS r ON l.st_id = r.st_id Where test_grp = 'pilot') AS pilot_converted,
    (Select uniqExact(st_id) From default.final_project_check AS l Join default.studs AS r ON l.st_id = r.st_id Where test_grp = 'control') AS control_converted,
    (Select uniqExact(st_id) From (Select st_id, sum(correct) AS num_of_peas From default.peas Where subject = 'Math' Group by st_id) Where num_of_peas >= 2) AS all_active_math_studs,
    (Select uniqExact(st_id) From (Select st_id, sum(correct) AS num_of_peas From default.peas AS l Join default.studs AS r ON l.st_id = r.st_id Where subject = 'Math' AND test_grp = 'pilot' Group by st_id) Where num_of_peas >= 2) AS pilot_active_math_studs,
    (Select uniqExact(st_id) From final_project_check Where subject = 'Math') AS all_converted_math_users,
    (Select uniqExact(st_id) From final_project_check AS l Join default.studs AS r ON l.st_id = r.st_id Where subject = 'Math' AND test_grp = 'pilot') AS pilot_converted_math_users

Select ROUND((3395000 / all_pilot_users), 2) AS pilot_ARPU,
    ROUND(((full_rev - 3395000) / all_control_users), 2) AS control_ARPU,
    ROUND((2855000 / uniqExact(l.st_id)), 2) AS pilot_ARPAU,
    ROUND((1320000 / (all_active_studs - uniqExact(l.st_id))), 2) AS control_ARPAU,
    ROUND((pilot_converted / all_pilot_users), 2) AS pilot_CR,
    ROUND((control_converted / all_control_users), 2) AS control_CR,
    ROUND((25 / uniqExact(l.st_id)), 2) AS pilot_active_CR,
    ROUND((14 / (all_active_studs - uniqExact(l.st_id))), 2) AS control_active_CR,
    ROUND((pilot_converted_math_users / pilot_active_math_studs), 2) AS pilot_active_math_CR,
    ROUND(((all_converted_math_users - pilot_converted_math_users) / (all_active_math_studs - pilot_active_math_studs)), 2) AS control_active_math_CR
    
    
FROM
    (Select 
        st_id as st_id,
        Sum(correct) as num_of_peas
    From default.peas
    
    GROUP BY st_id
    ) AS l
Join 
    (
    Select l.st_id, l.test_grp, r.money
    From default.studs AS l
    left Join default.final_project_check AS r ON l.st_id = r.st_id
    ) AS r ON l.st_id = r.st_id
Where num_of_peas > 10 AND test_grp='pilot'
'''
task_2_2 = ph.read_clickhouse(query_2_2, connection=connection_default)
task_2_2

,pilot_ARPU,control_ARPU,pilot_ARPAU,control_ARPAU,pilot_CR,control_CR,pilot_active_CR,control_active_CR,pilot_active_math_CR,control_active_math_CR
0,11508.47,4540.98,29739.58,10393.7,0.11,0.05,0.26,0.11,0.38,0.16


### Проверим корректность вычислений, скачав данные в датасеты и обработав их на python
### Let us check wether the calculations are accurate by downloading data into datasets and processing it via python

In [5]:
studs_query = '''
Select *
From default.studs
'''
peas_query = '''
Select st_id, correct, subject
From default.peas
'''
final_project_check_query = '''
Select st_id, money, subject
From default.final_project_check
'''

studs = ph.read_clickhouse(studs_query, connection=connection_default)
peas = ph.read_clickhouse(peas_query, connection=connection_default)
final_project_check = ph.read_clickhouse(final_project_check_query, connection=connection_default)

In [6]:
studs

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot
3,104885,pilot
4,104966,pilot
...,...,...
595,988626,pilot
596,993249,pilot
597,994983,pilot
598,996911,control


In [7]:
final_project_check

,st_id,money,subject
0,101432,85000,Math
1,101432,65000,Vizualization
2,104885,65000,Vizualization
3,104885,75000,Statistics
4,106464,85000,Math
...,...,...,...
56,969595,65000,Vizualization
57,969595,75000,Statistics
58,969595,100000,Python
59,976866,85000,Math


In [8]:
peas

,st_id,correct,subject
0,100379,1,Theory of probability
1,100379,0,Vizualization
2,100379,1,Theory of probability
3,100379,1,Vizualization
4,100379,1,Theory of probability
...,...,...,...
13240,996911,1,Statistics
13241,996911,0,Statistics
13242,996911,0,Statistics
13243,996911,1,Statistics


In [9]:
control_rev = final_project_check.merge(right=studs, on='st_id', how='left').query('test_grp == "control"').money.sum()
pilot_rev = final_project_check.merge(right=studs, on='st_id', how='left').query('test_grp == "pilot"').money.sum()
print('Pilot ARPU =', pilot_rev / studs.query('test_grp == "pilot"').shape[0])
print('Control ARPU =', control_rev / studs.query('test_grp == "control"').shape[0])

Pilot ARPU = 11508.474576271186
Control ARPU = 4540.983606557377


In [16]:
active_users_pilot = peas.groupby('st_id', as_index=False) \
    .agg({'correct': 'sum'}) \
    .merge(right=studs, on='st_id', how='left') \
    .query('correct > 10 and test_grp == "pilot"')

active_users_control = peas.groupby('st_id', as_index=False) \
    .agg({'correct': 'sum'}) \
    .merge(right=studs, on='st_id', how='left') \
    .query('correct > 10 and test_grp == "control"')

for_active_arpau_and_cr = peas.groupby('st_id', as_index=False) \
    .agg({'correct': 'sum'}) \
    .query('correct > 10') \
    .merge(right=final_project_check.groupby('st_id').agg({'money': 'sum'}), on='st_id', how='left') \
    .fillna(0.0) \
    .merge(right=studs, on='st_id', how='left')
for_active_arpau_and_cr

,st_id,correct,money,test_grp
0,100379,27,0.0,pilot
1,101432,14,150000.0,control
2,104966,26,0.0,pilot
3,106464,30,85000.0,pilot
4,111336,19,0.0,control
...,...,...,...,...
218,983096,14,0.0,pilot
219,98637,40,0.0,control
220,986680,19,0.0,control
221,994983,53,0.0,pilot


In [18]:
pilot_active_rev = for_active_arpau_and_cr.query('test_grp == "pilot"').money.sum()
print('pilot_active_rev = ', pilot_active_rev)

control_active_rev = for_active_arpau_and_cr.query('test_grp == "control"').money.sum()
print('control_active_rev = ', control_active_rev)

pilot_active_converted = for_active_arpau_and_cr.query('test_grp == "pilot" and money != 0.0').st_id.nunique()
print('pilot_active_converted = ', pilot_active_converted)

control_active_converted = for_active_arpau_and_cr.query('test_grp == "control" and money != 0.0').st_id.nunique()
print('control_active_converted = ', control_active_converted)

pilot_active_rev =  2855000.0
control_active_rev =  1320000.0
pilot_active_converted =  25
control_active_converted =  14


In [20]:
print('Pilot ARPAU =', pilot_active_rev / active_users_pilot.shape[0])
print('Control ARPAU =', control_active_rev / active_users_control.shape[0])
print()
print('Pilot active CR =', pilot_active_converted / active_users_pilot.shape[0])
print('Control active CR =', control_active_converted / active_users_control.shape[0])

Pilot ARPAU = 29739.583333333332
Control ARPAU = 10393.700787401574

Pilot active CR = 0.2604166666666667
Control active CR = 0.11023622047244094


In [12]:
active_math_users_pilot = peas.groupby(['st_id', 'subject'], as_index=False) \
    .agg({'correct': 'sum'}) \
    .merge(right=studs, on='st_id', how='left') \
    .query('correct >= 2 and test_grp == "pilot" and subject == "Math"')
active_math_users_control = peas.groupby(['st_id', 'subject'], as_index=False) \
    .agg({'correct': 'sum'}) \
    .merge(right=studs, on='st_id', how='left') \
    .query('correct >= 2 and test_grp == "control" and subject == "Math"')
pilot_math_conv = final_project_check.merge(right=studs, on='st_id', how='left').query('test_grp == "pilot" and subject == "Math"')
control_math_conv = final_project_check.merge(right=studs, on='st_id', how='left').query('test_grp == "control" and subject == "Math"')

print('Pilot active math CR = ', pilot_math_conv.shape[0] / active_math_users_pilot.shape[0])
print('Control active math CR = ', control_math_conv.shape[0] / active_math_users_control.shape[0])

Pilot active math CR =  0.38095238095238093
Control active math CR =  0.16326530612244897
